In [72]:
# %load_ext autoreload
# %autoreload 2

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

import re
import glob #for assembling files 
import wandb
import emoji

import sys
sys.path.insert(1, '../python')

from cb_preprocess import load_data, preprocess, tokenize, label_to_numeric
from nltk import ngrams
from nltk.corpus import stopwords
from collections import Counter

import preprocessor as pre
import fasttext as ft

from tokenizers import Tokenizer, normalizers
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import WordPiece as WP
from tokenizers.normalizers import NFD, StripAccents, Lowercase
from tokenizers.trainers import WordPieceTrainer

In [38]:
df = load_data(path = "../../data_wang/cyberbullying_tweets.csv")
df["y"] = label_to_numeric(df)
df["tweet_text"] = preprocess(df.tweet_text)
df["tokens"] = [" ".join(tokens) for tokens in tokenize(df.tweet_text)]

df

,tweet_text,label,lang,y,tokens
0,Here at home. Neighbors pick on my family and ...,age,en,0,here at home . neighbors pick on my family and...
1,Being bullied at school: High-achieving boys u...,age,en,0,being bullied at school : high - achieving boy...
2,There was a girl in my class in 6th grade who ...,age,en,0,there was a girl in my class in 6th grade who ...
3,He’s probably a white gay kid from some suburb...,age,en,0,he ’ s probably a white gay kid from some subu...
4,You are pushed ti resorting. Treating thr bull...,age,en,0,you are pushed ti resorting . treating thr bul...
...,...,...,...,...,...
47995,Can you imagine if Christians came together li...,religion,en,5,can you imagine if christians came together li...
47996,So how to support justice from the initial pro...,religion,en,5,so how to support justice from the initial pro...
47997,$RESERVED$ $MENTION$: If you harbour any doubt...,religion,en,5,$RESERVED$ $MENTION$ : if you harb ##our any d...
47998,"$MENTION$ $MENTION$ One thing about Muslims, t...",religion,en,5,"$MENTION$ $MENTION$ one thing about muslims , ..."


In [39]:
class LanguageIdentification:

    def __init__(self):
        pretrained_lang_model = "../fasttext_binaries/lid.176.bin"
        self.model = ft.load_model(pretrained_lang_model)

    def predict_lang(self, text):
        predictions = self.model.predict(text, k=2) # returns top 2 matching languages
        return predictions

In [40]:
language_identifer = LanguageIdentification()
lang_predictions = df.tweet_text.apply(language_identifer.predict_lang)

In [83]:
df["lang"] = lang_predictions.apply(lambda x: x[0][0][-2:])

df.groupby("lang").count()
df.to_csv("../../data_wang/cyberbullying_tweets.csv", index=True)

In [82]:
df

,tweet_text,label,lang,y,tokens,tweets
0,Here at home. Neighbors pick on my family and ...,age,en,0,here at home . neighbors pick on my family and...,here at home neighbors pick on my family and ...
1,Being bullied at school: High-achieving boys u...,age,en,0,being bullied at school : high - achieving boy...,being bullied at school high achieving boys u...
2,There was a girl in my class in 6th grade who ...,age,en,0,there was a girl in my class in 6th grade who ...,there was a girl in my class in 6th grade who ...
3,He’s probably a white gay kid from some suburb...,age,en,0,he ’ s probably a white gay kid from some subu...,he s probably a white gay kid from some suburb...
4,You are pushed ti resorting. Treating thr bull...,age,en,0,you are pushed ti resorting . treating thr bul...,you are pushed ti resorting treating thr bull...
...,...,...,...,...,...,...
47995,Can you imagine if Christians came together li...,religion,en,5,can you imagine if christians came together li...,can you imagine if christians came together li...
47996,So how to support justice from the initial pro...,religion,en,5,so how to support justice from the initial pro...,so how to support justice from the initial pro...
47997,$RESERVED$ $MENTION$: If you harbour any doubt...,religion,en,5,$RESERVED$ $MENTION$ : if you harb ##our any d...,if you harbour any doubts about what musl...
47998,"$MENTION$ $MENTION$ One thing about Muslims, t...",religion,en,5,"$MENTION$ $MENTION$ one thing about muslims , ...",one thing about muslims they want to exterm...


In [79]:
run = wandb.init(project='capstone')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ayushkumar (cyberbullying). Use `wandb login --relogin` to force relogin


In [84]:
artifact = wandb.Artifact("tweets", type="dataset")
artifact.add_dir("../../data_wang/")
wandb.log_artifact(artifact, aliases = ["preprocessed_tweets"])

wandb: Adding directory to artifact (.\..\..\data_wang)... Done. 0.1s


Error: You must call wandb.init() before wandb.log_artifact()

In [171]:
artifact = wandb.Artifact("wp_pretrained_tweet", type = "tokenizer")
artifact.add_file("../wp_tokenizer.json")
wandb.log_artifact(artifact)

In [173]:
table = wandb.Table(dataframe = df)
run.log({"data": table})

In [81]:
wandb.finish()

wandb: ERROR Control-C detected -- Run data was not synced
